In [3]:
from food_dataset import FoodDataset
from food_model import FoodModel
from train_settings import TrainSettings
from training_loop import TrainingLoop
import constants
from matplotlib import pyplot as plt


dt_train = FoodDataset(constants.dataset_root,
                      constants.class_names[:2], 
                      constants.class_labels[:2],
                      0,
                      10,
                      constants.transform)

dt_val = FoodDataset(constants.dataset_root,
                        constants.class_names[:2], 
                        constants.class_labels[:2],
                        10,
                        15,
                        constants.transform)

dt_test = FoodDataset(constants.dataset_root,
                        constants.class_names[:2], 
                        constants.class_labels[:2],
                        96,
                        32,
                        constants.transform)
model = FoodModel(10)

settings = TrainSettings("trying_out",
                         model,
                         dt_train,
                         dt_val,
                         epochs=10,
                         eval_after_epoch=1,
                        batch_size=2)
                        
training = TrainingLoop(settings)

training.train()



Epoch 1/10, Train Loss: 2.4373, Validation Loss: 2.4419 Evaluations: []


KeyboardInterrupt: 

In [ ]:
def show_image(image, label):
    plt.figure()
    plt.imshow(image.permute(1, 2, 0))
    plt.title(f"Label: {label}")
    
show_image(dt_train[799][0],dt_train.get_class_label(dt_train[799][1]))
show_image(dt_val[0][0],dt_val.get_class_label(dt_val[0][1]))

(8000, 2000, 1000)

In [4]:
len(dt_train), len(dt_val), len(dt_test)

(100, 150, 320)

In [3]:
sum([s.numel() for s in model.parameters()])

271048714